## [Python Programming for Biologists, Tel-Aviv University / 0411-3122 / Spring 2016](http://py4life.github.io/TAU2016/)
# Homework 4
### General instructions
- In questions 1 you will have to read and parse files. Download them [here](https://github.com/Py4Life/TAU2016/raw/master/files_for_hw4.rar), extract the rar file and then save the directory to your course directory (or wherever you access the notebooks from).
- In questions 2-3 you will have to use regex expressions. Please do your best in solving them!

## 1) Parsing $\lambda$ phage FASTQ
The FASTQ file format is commonly used to store deep sequencing reads data. It is similar to the FASTA format, but includes additional information. Each record is represented by four lines:
- Line 1 begins with a `@` character and is followed by a sequence identifier and an optional description (like a FASTA title line).
- Line 2 is the raw sequence letters.
- Line 3 is just a `+` character
- Line 4 encodes the quality values for the sequence in Line 2, and must contain the same number of symbols as letters in the sequence.  
Sequence quality is encoded with characters from:
```
!"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~
```
Where `!` represents the lowest quality, equivalent to a score of 1, and `~` is the highest quality with a score of 94.  

The function given below translates the characters into their corresponding scores and returns a dictionary which you can use later. Make sure you understand how to work with this dictionary before proceeding. Note that we use the \ character inside the string to include the quotation mark that follows it inside the string and not as a string ending (i.e., if we used only " instead of \", the quotation mark tells python that this is the end of the string).

In [1]:
def creates_scores_dict():
    scores_string = "!\"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~"
    scores_dict = {}
    for i,symbol in enumerate(scores_string):
        scores_dict[symbol] = i + 1
    return scores_dict

scores_dict = creates_scores_dict()
print(scores_dict)

{'5': 21, '"': 2, 'D': 36, '3': 19, '$': 4, '`': 64, ';': 27, 'o': 79, '?': 31, '@': 32, '4': 20, 'X': 56, 'F': 38, 'I': 41, 'P': 48, 'M': 45, '+': 11, 'l': 76, 'y': 89, '&': 6, '0': 16, 'Y': 57, ')': 9, '!': 1, '8': 24, 'k': 75, "'": 7, 'K': 43, '2': 18, 't': 84, '|': 92, '6': 22, '\\': 60, 'b': 66, 'm': 77, 'u': 85, 's': 83, '{': 91, 'N': 46, 'h': 72, ',': 12, 'A': 33, 'i': 73, 'd': 68, 'U': 53, 'W': 55, 'O': 47, '7': 23, 'p': 80, 'Q': 49, 'n': 78, '~': 94, '*': 10, 'g': 71, '(': 8, 'E': 37, ':': 26, 'v': 86, 'e': 69, 'R': 50, '=': 29, 'G': 39, 'B': 34, 'j': 74, 'f': 70, '^': 62, '#': 3, 'S': 51, '}': 93, ']': 61, '<': 28, 'c': 67, '[': 59, 'q': 81, '/': 15, '%': 5, '_': 63, 'L': 44, 'r': 82, 'V': 54, 'Z': 58, 'C': 35, '>': 30, 'J': 42, '-': 13, 'x': 88, '1': 17, 'H': 40, '.': 14, '9': 25, 'w': 87, 'z': 90, 'a': 65, 'T': 52}


The file `files_for_hw4/lambda_reads.fq` contains 10,000 reads from the sequencing of $\lambda$ phage. We would like to discard low quality reads. A low quality read is defined as one with a mean score lower than some predefined threshold.  

**a)** Write a function `mean_score` that receives a read quality string and returns the mean score (float) of the read. 

For example, the quality string `!!!!!` is equivalent to the scores `1,1,1,1,1`, and thus the mean is `1.0`. 
However, the string `49@5<*>=E` is equivalent to the scores `20,25,32,21,28,10,30,29,37` and has a mean of `25.77`.

In [2]:
def mean_score(read_quality_string):
    sum = 0
    scores_dict = creates_scores_dict() #we need to call the dictionary to have it initiated
    for char in read_quality_string:
        sum += scores_dict[char]
    return sum/len(read_quality_string)
    
    
assert(mean_score('!!!!!') == 1.0)
assert(round(mean_score('49@5<*>=E'),2) == 25.78)

**b)** Write a function `parse_FASTQ` that parses a FASTQ file. It receives a path to a FASTQ file on the local filesystem and returns a _dictionary_ where keys are sequences and values are the mean scores of the sequences. 

Use the function on the provided file `files_for_hw4/lambda_reads.fq`.

It is recommended to use the `readline()` method of file objects (although other solutions are also possible).
Remeber to remove hidden characters in the end of the lines with str.strip().

In [5]:
def parse_FASTQ(file):
    with open(file) as f: #by default, open() uses 'r' argument (read), so we don't have to write it
        quality_dict = {}
        last_line = ""
        line = f.readline()
        while line: 
            # here we can use for loop, but usually when iterating over an object - do not change it inside the loop

            line = line.strip()
            #attention! a quality score line can start with '@' - so we have to distinguish between the
            # two types of lines. We know that the quality score line comes after a '+' line
            
            if line.startswith('@') and last_line != "+":
                # it's a sequence identifier line - so the next one is the sequence
                line = f.readline().strip()
                seq = line
                
            elif last_line == "+":
                #we reached a quality score line
                quality_dict[seq] = mean_score(line)
                
            last_line = line
            line = f.readline()
    return quality_dict
    
    
# parse lambda reads file  
lambda_reads_file = "files_for_hw4/lambda_reads.fq"
lambda_seqs_dict = parse_FASTQ(lambda_reads_file)

#example
print(lambda_seqs_dict["ATTGAACAAATTAACATCGCTCTGGAGCAAAAAGGGTCCNGGAATTTGTCAGCCTGGGTCA"])
print(mean_score("7CD0D/1G9!9/H'482-E\",G:,6A;:>34-)5;44B&8-$D8260#\".$:>=?%7D\":8"))

20.852459016393443
20.852459016393443


**c)** Write a function `filter_reads` that takes the output from section **b** and a score cutoff (integer/float) and prints out the sequences with scores higher than the cutoff. 

Each sequence will be printed in a separate line (no need to keep the FASTQ format). Try different cutoffs (5,10,20) on the $\lambda$ phage reads.

In [14]:
def filter_reads(seqs_dict,cutoff,out_file):
    # your code here. remove the pass statement.
    with open(out_file, "w") as fpw:
        for seq in seqs_dict:
            if seqs_dict[seq]>=cutoff:
                print(seq, file=fpw)
    
    
# run on Lambda reads
lambda_filtered_file = "files_for_hw4/lambda_filtered_reads.txt"
filter_reads(lambda_seqs_dict, 25.8, lambda_filtered_file)

## 2) Regex drills
In this question, you don't have to write real code, just write the regular expression you'd use within the quotation marks.  

* It is highly recommended to use [The regex Coach](http://www.weitz.de/regex-coach/) for this question.

__Reminder from the lecture:__
We can put any list of characters within the brackets. There are also a few tricks to make things easier:  
* [0-9] - any digit
* [a-z] - any letter
* [a-p] - any letter between a and p
  
There are also special symbols for common groups:  
* \d - any digit (equivalent to [0-9])
* \w - any 'word' character - letters, digits and underscore (equivalent to [a-zA-Z0-9\_)
* \s - any whitespace character - space, tab, newline and other weird stuff (equivalent to [ \t\n\r\f\v])
  
And finally, there's the _wildcard_ symbol, represented by a dot (.).  
This means any character (except for a newline).  
__Careful with this one!__ It'll take almost anything, so use it wisely.


a) Write a regex that will match strings containing any kind of number: positive/negative, integer/float etc. For example, all of the following should be matched: 7 , -3 , 6.14 , -0.00054.
* In the lecture we gave one example of representing digits, i.e., "[0-9]". You can also use \d within the string to represnt numbers. Try it!
* If you want to treat repetitive characters, you can use '\*' or '+' after the character. For example, multiple digits can be found with '\d+'. '*' represents none or multiple repetitions, and '+' represents at least 1 repetition.

In [22]:
import re

pattern1 = re.compile(r'^-?\d+(\.\d+)?$') 
#NOTE: ^,$ signs (begining, ending) are not necessary here - because we asked a number that is found 
#      within a string, but use them for practice: we need them for validating 
#      that the pattern matches the whole string and not found within it. 
#      For example, the pattern could be found within 7.14.6 - but this is not a valid number

#we have here:
# -?   maybe there's a minus sign at the begining
# \d+  at least one digit (the integer part of the number)

# the next two are enclosed by ()? because itmight be a decimal number, 
#    so there might be a floating point. And if so, there must be numbers after the floating point
# \.   the floating point (if there's no \ before the dot, it could match any character by the regex rules!) 
# \d+  after the floating there must be additional digits

assert(re.search(pattern1, "7"))
assert(re.search(pattern1, "-14"))
assert(re.search(pattern1, "-0.0005"))
assert(re.search(pattern1, "79.12"))

assert(re.search(pattern1, "7,") is None)
assert(re.search(pattern1, "7t5") is None)
assert(re.search(pattern1, ".9") is None)
assert(re.search(pattern1, "99.5.9") is None)

b) Write a regex that will match strings that __end__ with a number between 100 and 199, followed by a '.' __or__ a '\' character.
* Enforcing a regex at the end of the string can be done with assigning a dollar sign. For example, the pattern 'xyz\$' can be found within 'qvwxyz' but not within 'xyza'.

In [24]:
pattern2 = re.compile(r'1\d{2}$')

# the $ sign in the end demands that the string ends with pattern
# before that, there must be a '1' and then another digit (represented by '\d') twice (therefore '{2}')

assert(re.search(pattern2, "blabla104"))
assert(re.search(pattern2, "blabla1111104"))

assert(re.search(pattern2, "104blabla") is None)
assert(re.search(pattern2, "blabla1200") is None)

c) Write a regex that will match whole strings of prices in dollars, such as '100\$', '2.99\$', '500.90\$', but not '7.656\$', '80.0001\$' or 'price is: 56.80\$'.

In [31]:
pattern3 = re.compile(r'^\d+(\.\d{2})?\$$')

#similar to pattern1, small differences to fit the request

assert(re.search(pattern3, "100$"))
assert(re.search(pattern3, "5445.50$"))
assert(re.search(pattern3, "0.99$"))
assert(re.search(pattern3, ".99$") is None)
assert(re.search(pattern3, "7.656$") is None)
assert(re.search(pattern3, "100") is None)
assert(re.search(pattern3, "the price is: 100$") is None)

d) Write a regex that will match strings beginning with 3 to 8 uppercase letters, followed by at least 4 characters, which can be anything but '%' or '!', and end with 'XY' or 'QW'. 
* To represent that a character CANNOT be within a string, we'll put it inside square paranthesis with '^' before it. For example, if we want to find a sequence ATNTGCTG that does not contain 'A' in the third position, we'll use the pattern 'AT[^A]TGCTG'.

In [46]:
pattern4 = re.compile(r'^[A-Z]{3,8}[^%!](XY|QW)$')

assert(re.search(pattern4, "ABC7QW"))
assert(re.search(pattern4, "ZZZZZZZXY")) #note here that [^%!] matches Z
assert(re.search(pattern4, "HFKVRITKV6QW") is None)
assert(re.search(pattern4, "ABC6XYQW") is None)
assert(re.search(pattern4, "ABC6XQW") is None)
assert(re.search(pattern4, "ABC!XY") is None)


## 3) Gene names

The code below includes a list of made-up gene names. Complete it to only print gene names that satisfy the following criteria:  
1. Contain the letter 'd' __or__ 'e'  
2. Contain the letter 'd' __and__ 'e', in that order (not necessarily adjacent)
3. Contain three or more digits one after the other. A pattern for finding a specific number of repetitions is by '{min,max}', for exaample, 'a{5,10}' will match a string with 5 to 10 repetitions of 'a'.

In [47]:
import re
genes = ['xkn59438', 'yhdck2', 'eihd39d9', 'chdsye847', 'hedle3455', 'xjhd53e', '45da', 'de37dp','map492ty']

# 1.
print('Gene names containing d or e:')
regex1 = re.compile(r'd|e')
for gene in genes:
    if re.search(regex1, gene):
        print(gene)
    
        
print('------------------------')

# 2.
print('Gene names containing d and e, in that order:')
regex2 = re.compile(r'd.*e')
for gene in genes:
    if re.search(regex2, gene):
        print(gene)

        
print('------------------------')

# 3.
print('Gene names containing three digits in a row:')
regex3 = re.compile(r'\d{3}')
for gene in genes:
    if re.search(regex3, gene):
        print(gene)


Gene names containing d or e:
yhdck2
eihd39d9
chdsye847
hedle3455
xjhd53e
45da
de37dp
------------------------
Gene names containing d and e, in that order:
chdsye847
hedle3455
xjhd53e
de37dp
------------------------
Gene names containing three digits in a row:
xkn59438
chdsye847
hedle3455
map492ty
